# Option A: Deploy Everything New

Use this notebook when starting fresh with no existing Azure resources. This will deploy:

- Storage Account
- Cosmos DB (serverless)
- Document Intelligence
- Synapse Analytics Workspace
- Key Vault
- Function App + App Service Plan
- Log Analytics Workspace

**Estimated deployment time:** 10-15 minutes

## Step 1: Configure Variables

Update the following variables with your values:

In [ ]:
# ============================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================

$SUBSCRIPTION_ID = "<YOUR_SUBSCRIPTION_ID>"
$LOCATION = "eastus"
$ENVIRONMENT = "dev"  # or "prod"

# SQL Administrator password for Synapse
# Must meet complexity requirements: 8+ chars, uppercase, lowercase, number, special char
$SQL_ADMIN_PASSWORD = "<YOUR_SECURE_PASSWORD>"

# ============================================

# Set subscription
az account set --subscription $SUBSCRIPTION_ID

Write-Host "Configuration set:" -ForegroundColor Green
Write-Host "  Subscription: $SUBSCRIPTION_ID"
Write-Host "  Location: $LOCATION"
Write-Host "  Environment: $ENVIRONMENT"

## Step 2: Preview Deployment (What-If)

Always preview what will be deployed before actually deploying.

In [ ]:
# Navigate to project root
Set-Location -Path "$PSScriptRoot/.."

# Select parameter file based on environment
$PARAM_FILE = "infra/parameters/$ENVIRONMENT.bicepparam"

Write-Host "Running What-If deployment preview..." -ForegroundColor Cyan
Write-Host "Parameter file: $PARAM_FILE" -ForegroundColor Yellow

az deployment sub what-if `
  --subscription $SUBSCRIPTION_ID `
  --location $LOCATION `
  --template-file infra/main.bicep `
  --parameters $PARAM_FILE `
  --parameters sqlAdministratorPassword=$SQL_ADMIN_PASSWORD

## Step 3: Deploy Infrastructure

Deploy all Azure resources. The resource group will be created automatically.

In [ ]:
Write-Host "Deploying infrastructure..." -ForegroundColor Cyan
Write-Host "This may take 10-15 minutes..." -ForegroundColor Yellow

$deployment = az deployment sub create `
  --subscription $SUBSCRIPTION_ID `
  --location $LOCATION `
  --template-file infra/main.bicep `
  --parameters $PARAM_FILE `
  --parameters sqlAdministratorPassword=$SQL_ADMIN_PASSWORD `
  --output json | ConvertFrom-Json

if ($LASTEXITCODE -eq 0) {
    Write-Host "Deployment completed successfully!" -ForegroundColor Green
} else {
    Write-Host "Deployment failed. Check the error above." -ForegroundColor Red
}

## Step 4: Get Deployment Outputs

Retrieve the deployed resource information for local development and function deployment.

In [ ]:
# Get deployment outputs
Write-Host "Retrieving deployment outputs..." -ForegroundColor Cyan

$outputs = az deployment sub show `
  --subscription $SUBSCRIPTION_ID `
  --name main `
  --query properties.outputs `
  --output json | ConvertFrom-Json

# Display key values
Write-Host "`nDeployment Outputs:" -ForegroundColor Green
Write-Host "==================" -ForegroundColor Green

Write-Host "`nResource Group: $($outputs.resourceGroupName.value)"
Write-Host "Function App: $($outputs.functionAppName.value)"
Write-Host "Storage Account: $($outputs.storageAccountName.value)"
Write-Host "Cosmos DB: $($outputs.cosmosAccountName.value)"
Write-Host "Document Intelligence: $($outputs.documentIntelligenceName.value)"
Write-Host "Key Vault: $($outputs.keyVaultName.value)"
Write-Host "Synapse Workspace: $($outputs.synapseWorkspaceName.value)"

# Store for later use
$RESOURCE_GROUP = $outputs.resourceGroupName.value
$FUNC_APP_NAME = $outputs.functionAppName.value
$STORAGE_ACCOUNT = $outputs.storageAccountName.value

## Step 5: Deploy Function Code

Deploy the Python function code to the newly created Function App.

In [ ]:
# Get Function App name from deployment output if not already set
if (-not $FUNC_APP_NAME) {
    $FUNC_APP_NAME = az deployment sub show `
      --subscription $SUBSCRIPTION_ID `
      --name main `
      --query properties.outputs.functionAppName.value `
      --output tsv
}

Write-Host "Deploying function code to: $FUNC_APP_NAME" -ForegroundColor Cyan

# Navigate to functions directory
Set-Location -Path "$PSScriptRoot/../src/functions"

# Deploy function code
func azure functionapp publish $FUNC_APP_NAME --python

# Return to project root
Set-Location -Path "$PSScriptRoot/.."

## Step 6: Verify Deployment

Test that the Function App is responding correctly.

In [ ]:
# Get Function App URL
$FUNC_URL = "https://$FUNC_APP_NAME.azurewebsites.net"

Write-Host "Testing health endpoint..." -ForegroundColor Cyan
Write-Host "URL: $FUNC_URL/api/health" -ForegroundColor Yellow

try {
    $response = Invoke-RestMethod -Uri "$FUNC_URL/api/health" -Method Get
    Write-Host "`nHealth check response:" -ForegroundColor Green
    $response | ConvertTo-Json
} catch {
    Write-Host "`nHealth check failed: $_" -ForegroundColor Red
    Write-Host "Note: The function may take a few minutes to start up after deployment." -ForegroundColor Yellow
}

## Step 7: Deploy Synapse Artifacts

Deploy pipelines, linked services, and datasets to Synapse workspace.

In [ ]:
# Get resource names from deployment
$outputs = az deployment sub show `
  --subscription $SUBSCRIPTION_ID `
  --name main `
  --query properties.outputs `
  --output json | ConvertFrom-Json

$SYNAPSE_WORKSPACE = $outputs.synapseWorkspaceName.value
$STORAGE_ACCOUNT = $outputs.storageAccountName.value
$FUNC_APP_NAME = $outputs.functionAppName.value
$KEY_VAULT_NAME = $outputs.keyVaultName.value
$RESOURCE_GROUP = $outputs.resourceGroupName.value

# Build URLs
$STORAGE_URL = "https://$STORAGE_ACCOUNT.blob.core.windows.net"
$FUNC_URL = "https://$FUNC_APP_NAME.azurewebsites.net"
$KV_URL = "https://$KEY_VAULT_NAME.vault.azure.net/"

Write-Host "Deploying Synapse artifacts..." -ForegroundColor Cyan
Write-Host "  Workspace: $SYNAPSE_WORKSPACE"
Write-Host "  Storage URL: $STORAGE_URL"
Write-Host "  Function URL: $FUNC_URL"
Write-Host "  Key Vault URL: $KV_URL"

# Navigate to project root
Set-Location -Path "$PSScriptRoot/.."

# Run deployment script
.\scripts\Deploy-SynapseArtifacts.ps1 `
  -WorkspaceName $SYNAPSE_WORKSPACE `
  -ResourceGroup $RESOURCE_GROUP `
  -DeploymentMode direct `
  -StorageAccountUrl $STORAGE_URL `
  -FunctionAppUrl $FUNC_URL `
  -KeyVaultUrl $KV_URL

## Step 8: Store Function Key in Key Vault

The Synapse pipeline needs the Function App's host key to authenticate. Store it in Key Vault.

In [ ]:
# Get Function App host key
Write-Host "Getting Function App host key..." -ForegroundColor Cyan

$FUNCTION_KEY = az functionapp keys list `
  --name $FUNC_APP_NAME `
  --resource-group $RESOURCE_GROUP `
  --query "functionKeys.default" `
  --output tsv

if ($FUNCTION_KEY) {
    Write-Host "Function key retrieved successfully" -ForegroundColor Green
    
    # Store in Key Vault
    Write-Host "Storing function key in Key Vault..." -ForegroundColor Cyan
    
    az keyvault secret set `
      --vault-name $KEY_VAULT_NAME `
      --name "FunctionAppHostKey" `
      --value $FUNCTION_KEY `
      --output none
    
    Write-Host "Function key stored in Key Vault as 'FunctionAppHostKey'" -ForegroundColor Green
} else {
    Write-Host "Failed to retrieve function key" -ForegroundColor Red
}

## Step 9: Grant Synapse Access to Key Vault

Allow Synapse workspace's managed identity to read secrets from Key Vault.

In [ ]:
# Get Synapse managed identity
Write-Host "Getting Synapse managed identity..." -ForegroundColor Cyan

$SYNAPSE_IDENTITY = az synapse workspace show `
  --name $SYNAPSE_WORKSPACE `
  --resource-group $RESOURCE_GROUP `
  --query identity.principalId `
  --output tsv

Write-Host "Synapse Identity: $SYNAPSE_IDENTITY"

# Grant Key Vault access
Write-Host "Granting Key Vault access..." -ForegroundColor Cyan

az keyvault set-policy `
  --name $KEY_VAULT_NAME `
  --object-id $SYNAPSE_IDENTITY `
  --secret-permissions get list `
  --output none

Write-Host "Key Vault access granted to Synapse" -ForegroundColor Green

## Deployment Complete!

Your Azure Document Intelligence pipeline is now deployed. Next steps:

1. **Test the pipeline** - See `05-Synapse-Pipeline.ipynb`
2. **Set up local development** - See `03-Local-Development.ipynb`
3. **Configure analytics** - See `06-Analytics-SynapseLink.ipynb`

### Resource Summary

In [ ]:
# Display deployment summary
Write-Host "`n========================================" -ForegroundColor Cyan
Write-Host "   DEPLOYMENT SUMMARY" -ForegroundColor Cyan
Write-Host "========================================" -ForegroundColor Cyan

$outputs = az deployment sub show `
  --subscription $SUBSCRIPTION_ID `
  --name main `
  --query properties.outputs `
  --output json | ConvertFrom-Json

Write-Host "`nResource Group:       $($outputs.resourceGroupName.value)"
Write-Host "Function App:         https://$($outputs.functionAppName.value).azurewebsites.net"
Write-Host "Storage Account:      $($outputs.storageAccountName.value)"
Write-Host "Cosmos DB:            $($outputs.cosmosAccountName.value)"
Write-Host "Document Intelligence: $($outputs.documentIntelligenceName.value)"
Write-Host "Key Vault:            $($outputs.keyVaultName.value)"
Write-Host "Synapse Workspace:    $($outputs.synapseWorkspaceName.value)"
Write-Host "`n========================================" -ForegroundColor Cyan